- 大致上是用 selenium 套件連動 webdriver 的腳本來模擬FB登入並瀏覽，要注意webdriver對應瀏覽器的版本對不對，這邊我是用 chromedriver.exe
(參考：https://gist.github.com/HaoHsiu-Huang/f4a68bec77c17e0e118a6a5cb4dffed1)(有點舊，有些內容不適用)

- 最後會根據爬到的內容來決定要不要傳訊息，這邊是使用line notify的功能，要在額外申請     
(參考：https://www.learncodewithmike.com/2020/06/python-line-notify.html)

- 有時候job()會產生錯誤，content[0] 會有 out of list range的問題，不知道是怎麼樣我也不知道，我猜是腳本有時候跑不好的bug

In [1]:
# 載入需要的套件
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import schedule

In [2]:
#關通知
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values':{'notifications': 2}}
options.add_experimental_option('prefs', prefs)

#不顯示視窗 覺得視窗礙眼可以用
#options.add_argument("--headless")
#options.add_argument('--disable-gpu') 

# ------ 設定要前往的網址 ------
url = 'https://www.facebook.com'  

# ------ 登入的帳號與密碼 ------
username = ''
password = ''

# 找社團(二手拍，並按照最新貼文排序)
url_g = 'https://www.facebook.com/groups/817620721658179?sorting_setting=CHRONOLOGICAL'

#元素定位
poster_class = 'x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xi81zsa x1yc453h'
date_class = 'x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g xt0b8zv xo1l8bm'
contents_class = 'x1iorvi4 x1pi30zi x1l90r2v x1swvt13'

#文章暫存
temp = ''

In [3]:
#點擊要打開的東西
def click_name(driver,name):
    
    str = "//*[text()='%s']" % name
    more_button = driver.find_elements(By.XPATH,str)
    for i in range(len(more_button)):
        try:
            more_button[i].click()
        except:
            continue

#line 通知連動
def line_notify(content):
    headers = {
            "Authorization": "Bearer " + "09ZRoR5lAVH9PGVaqL7ae0rTXwiUnuRl5SBrsD7RKdE",
            "Content-Type": "application/x-www-form-urlencoded"
        }
    params = {"message": time.strftime("[ %m-%d %H:%M:%S ] ", time.localtime()) + '\n' + "二手拍有您所關注字詞的貼文，貼文如下：" + '\n' + content + '\n' + url_g}

    r = requests.post("https://notify-api.line.me/api/notify",headers=headers, params=params)
    print(r.status_code)  #200

#這個沒用到 用來輸出成exel的
def out_put(poster,date,contents):
   collect_fb = pd.DataFrame()
      
   df_fb = pd.DataFrame({
                      'person': pd.Series(poster),
                      'date': pd.Series(date),
                      'content': pd.Series(contents),
                     })
   collect_fb = pd.concat([collect_fb, df_fb], ignore_index=True)
   
   writer = pd.ExcelWriter('output1.xlsx', engine='xlsxwriter')
   collect_fb.to_excel(writer, sheet_name='fb')
   writer.close()

#滾動視窗 因為臉書要下滑才會顯示更多   
def rolling(count):
    # 往下滑
    for j in range(count):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(1)
    # 往回滑    
    driver.execute_script("window.scrollTo(0,-document.body.scrollHeight)")

In [4]:
# ------ 透過Browser Driver 開啟 Chrome ------
driver = webdriver.Chrome(options=options)  

# 最大化視窗 
driver.maximize_window()     

# ------ 前往該網址 ------
driver.get(url)    

# ------ 輸入賬號密碼 ------
driver.find_element(By.ID,'email').send_keys(username)
driver.find_element(By.ID,'pass').send_keys(password)        
driver.find_element(By.NAME,'login').click()
time.sleep(1)

# 找社團
driver.get(url_g)
time.sleep(1)

In [5]:
def job():
   
   driver.refresh()
   time.sleep(1)
   rolling(1)
   time.sleep(1)
   click_name(driver,'顯示更多')
   time.sleep(1)

   contents = []
   date = []
   poster = []
   articles = []

   htmltext = driver.page_source
   time.sleep(1)
   soup = BeautifulSoup(htmltext, 'html.parser')

   # 抓取文章
   articles = soup.find_all(class_ ='x9f619 x1n2onr6 x1ja2u2z x2bj2ny x1qpq9i9 xdney7k xu5ydu1 xt3gfkd xh8yej3 x6ikm8r x10wlt62 xquyuld')

   # 丟掉不是文章的
   i=0
   while True:
      if articles[i].find(class_ = contents_class) == None or articles[i].find(class_ = poster_class) == None or articles[i].find(class_ = date_class) == None:
         articles.pop(i)
         i=i-1
         if i+1 == len(articles):
            break
      i=i+1
         
   for a in articles:
      poster.append(a.find(class_ = poster_class).text)
      date.append(a.find(class_ = date_class).text)
      contents.append(a.find(class_ = contents_class).text)
      
   #driver.close()

   #這邊是只判斷第一篇文章 print過的就不print
   #有包含 '椅' 的內容會連動line通知
   global temp 
   if contents[0] != temp:
      if '椅' in contents[0]:
         line_notify(contents[0])
            
      temp = contents[0]   
      print(time.strftime("[ %m-%d %H:%M:%S ] ", time.localtime()) + '\n' + contents[0])
                   
   #out_put(poster,date,contents)


In [6]:
#這邊使用schedule 5~15秒隨機執行一次job 缺點是要一直開著執行
schedule.every(5).to(15).seconds.do(job)
while True:
    schedule.run_pending()

[ 06-09 19:34:18 ] 
【搬家出清】商品網址：https://tingjulee.notion.site/360bb9a34e9e4c14beb75d68......狀態、描述、更清晰的照片都在裡頭，清大交大可面交，也可linepay。歡迎私訊
[ 06-09 19:47:04 ] 
《售》AirPods 2代（全新未開封）幫室友賣的去年BTS方案拿到的 室友原本要留著用出售原因：他現在想換pro 故售出價格：3000
[ 06-09 19:49:39 ] 
Booking/Agoda飯店協助比價代訂便宜飯店疫情解封，假期住宿想比較看看有沒有更划算的價格嗎？因本身有Booking Genius跟Agoda白金會員，且想累積信用卡刷卡滿額禮。可協助板上有訂房需求者免費比價，代訂便宜飯店，最高折扣可達75折。1.請提供想比價的飯店入住日期、房型。2.將提供最便宜的報價（網站折扣完畢後最終顯示之價格，會提供截圖）。3.計價方式=（網頁上優惠價格）*（1+1.5%國外刷卡交易手續費）=（支付總額）3.確認欲訂購後，不退款房型請先匯款，免費取消房型，訂購後將提供訂購成功憑證，請於一天內匯款完成。4.如有取消需求依照訂房網規定，若已無法免費取消產生之費用請自行負擔。
[ 06-09 19:51:27 ] 
［賣二手遊戲］物品：PS5地平線價格：850面交地點：新竹交大
[ 06-09 20:01:07 ] 
畢業出清1.電火鍋爐使用過一次 $5002.炸醬麵4包 $1003.咖啡濾掛綜合包15包 $1004.剩1/2的洗衣精 $100希望在八舍附近面交
[ 06-09 20:08:47 ] 
《售》普通化學乙 黃暄益教授 指定用書（14版，章節跟教授上課進度一致）（附考古）500$普化乙也是電機系數理必選之一，黃教授的課學風自由，推薦d(^_^o)
[ 06-09 20:11:47 ] 
［售］  大碩資工考研用書（不拆賣）［內容］如照片，裡面所有的內容，還會再附贈很多考古題［說明］你是第三手，有我大量翻閱的微耗損，然後有些書的前面部分有上個主人的筆記，不過整體上書況還是挺不錯的，而且也幫你證明了認真讀完是可以上岸的，若真好奇書況可以再慢慢談談［售價］全部1500  交大十三舍面交［折扣遊戲！不能疊加！］認識的或是雄友會學弟妹  -300資工系系排名比65%糟的

KeyboardInterrupt: 